In [1]:
# Calculation of VAR considering a simulation of potential dynamics of future stock prices in a portfolio using steps
# 1. Determine dynamics of assets in portfolio
# 2. Apply Monte_Carlo to simulate value of assets n_days into the future
# 3. Determine portfolio value in each scenario
# 4. Determing alpha value of the projected values

In [8]:
# Import modules
import numpy as np
import numpy.matlib
import math
import random
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.stats import uniform


In [9]:
# Share information based on share dynamics example 3 stocks in portfolio
S_0_0   = 100 # first share price starting value
S_0_1   = 95  # second share price starting value
S_0_2   = 50  # third share price starting value
S_0     = np.array([[S_0_0],[S_0_1],[S_0_2]]) # array of initial start value assets in portfolio

# Volatilities of each share
sigma_0 = 0.15 # first share volatility
sigma_1 = 0.2  # second share volatility
sigma_2 = 0.3  # third share volatility
sigma   = np.array([[sigma_0], [sigma_1], [sigma_2]])

# Interdependancies asset values captured by correlation matrix 
corr_matrix = np.array([[1,0.2,0.4],[0.2,1,0.8],[0.4,0.8,1]])

# Cholesky decomposition LL' = corr_matrix
L = np.linalg.cholesky(corr_matrix)

# General information
r = 0.1 # risk free interest rate
T = 1   # considering a 1 year period

In [10]:
# Considerations and assumptions for VAR calculations
# Assume here stock prices follow geometric brownian motion - can use different
# Assume holdings of 1 units of asset - can assume something else
np.random.seed(0)
num_simulations      = 10000
alpha                = 0.05 #alpha value considered here is 5% (probablity)
curr_portfolio_value = np.sum(S_0) #current starting portfolio value

def terminal_share_price(S_0,r, sigma, T,Z):
    """
    model final stock prices based on geometric brownian motion, fixed holdings in each asset that dont change,
    and Z which in this case is correlated multivariate normal sample
    """
    return S_0*(np.exp((r-sigma**2/2)*T + sigma*np.sqrt(T)*Z))


In [12]:
# Calculation of Z the multivariate normal sample of values 
uncorr_normal_sample = norm.rvs(size = [3, num_simulations]) # N(0,1) variables for number simulations = num_simulations
corr_normal_sample   = np.matmul(L,uncorr_normal_sample)
Z                    = corr_normal_sample
np.shape(Z)

(3, 10000)

In [14]:
# Portfolio value future 
future_portfolio_value = np.sum(terminal_share_price(S_0,r,sigma,T,Z), axis = 0)
future_portfolio_value

array([223.37503515, 320.06612466, 314.37082555, ..., 316.58652258,
       272.41216774, 201.22465491])

In [22]:
# Calculation of VAR using Monte Carlo
# 1. Calculate expected portfolio returns from the simulations
# 2. Sort the returns to find smallest portfolio returns
# 3. Determin VAR as the alpha quantile in our case alpha = 0.05

portfolio_return = (future_portfolio_value - curr_portfolio_value) / curr_portfolio_value

# sorting portfolio returns
portfolio_return = np.sort(portfolio_return)

# Determination of VAR
alpha_percentile_pos  =  int(np.floor(alpha*num_simulations))-1 # position in array of portfolio returnes based on percentile
VAR_estimate          = -1* portfolio_return[alpha_percentile_pos]
VAR_estimate

0.15890346110434664